##Imports

In [1]:
!pip install gradio

In [2]:
!pip install catboost
!pip install scipy==1.14.0
!pip install optuna

  Using cached optuna-4.4.0-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.16.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
Using cached optuna-4.4.0-py3-none-any.whl (395 kB)
Using cached alembic-1.16.2-py3-none-any.whl (242 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)


In [3]:
import pandas as pd
import numpy as np
import json
import scipy
from scipy.optimize import nnls
from scipy.optimize import lsq_linear

import ast

import torch
import matplotlib.pyplot as plt

import torch
from tqdm import tqdm
import optuna

from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, explained_variance_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import gradio as gr

In [ ]:
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Data

---



---



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
vacancies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cost_of_skills/IT_vacancies_full.csv')

In [ ]:
# Applying the logic
vacancies['To'] = vacancies.apply(lambda row: row['From'] if pd.isna(row['To']) and not pd.isna(row['From']) and row['To'] != 0 else row['To'], axis=1)
vacancies['From'] = vacancies.apply(lambda row: row['To'] - 10000 if pd.isna(row['From']) and not pd.isna(row['To']) and row['To'] != 0 else row['From'], axis=1)

In [ ]:
vacancies = vacancies.rename(columns={'Keys': 'Skills'})
vacancies['Skills'] = vacancies['Skills'].apply(lambda x: ast.literal_eval(x))
vacancies['Profarea names'] = vacancies['Profarea names'].apply(lambda x: eval(x))
vacancies['Professional roles'] = vacancies['Professional roles'].apply(lambda x: eval(x))

vacancies['Specializations'] = vacancies['Specializations'].apply(lambda x: eval(x))

In [ ]:
# Applying the logic
vacancies['To'] = vacancies.apply(lambda row: row['From'] if pd.isna(row['To']) and not pd.isna(row['From']) and row['To'] != 0 else row['To'], axis=1)
vacancies['From'] = vacancies.apply(lambda row: row['To'] - 10000 if pd.isna(row['From']) and not pd.isna(row['To']) and row['To'] != 0 else row['From'], axis=1)

# Dropping rows where both From and To are NaN
vacancies.dropna(subset=['From', 'To'], how='all', inplace=True)

vacancies = vacancies[vacancies['To'] != 0]

#vacancies['Profarea names'] = vacancies['Profarea names'].apply(lambda x: eval(x))
vacancies = vacancies[vacancies['Salary'] == True]
vacancies = vacancies.drop('Salary', axis=1)

In [ ]:
vacancies = vacancies.reset_index()

## Stat

In [ ]:
def build_skill_dict(vacancies, prof_areas):
    """
    Функция пробегается по каждому названию области,
    обучает модель (получаем вектор x) и формирует итоговый словарь.
    """
    skill_dict = {}

    for area in prof_areas:
        # 1. Фильтруем вакансии, относящиеся к выбранной области
        data = vacancies[
            vacancies['Profarea names'].apply(
                lambda prof_list: any(area in prof for prof in prof_list)
            )
        ].copy().dropna().reset_index(drop=True)

        # 2. Минимальная предобработка (как в вашем коде):
        data = data[['Skills', 'From', 'To']].rename(columns={'From': 'from', 'To': 'to'})
        # Убираем строки, где from == to
        data = data[data['from'] != data['to']]
        # Убираем строки, где Skills пуст
        data = data[data['Skills'].apply(len) > 0]

        if len(data) == 0:
            # Если для этой области данных нет, создаём пустой словарь
            skill_dict[area] = {}
            continue

        # 3. Собираем список всех навыков
        all_skills = get_all_skills(data)

        # 4. Создаём словарь маппинга навыка -> индекс
        skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

        # 5. Собираем матрицу A
        A = get_matrix(data, skill_id_dict)
        A_torch = torch.tensor(A, dtype=torch.float32, device=device)

        U, S, Vh = torch.linalg.svd(A_torch, full_matrices=False)

        # Максимальное сингулярное число
        max_singular_value = torch.max(S)

        print(f"Специальность: {area} : {A_torch.shape[0]}, {A_torch.shape[1]}, собственные числа: {max_singular_value}")


In [ ]:
len(prof_areas)

28

In [ ]:
build_skill_dict(vacancies, prof_areas)

Специальность: Государственная служба, некоммерческие организации : 24, 118, собственные числа: 6.262829303741455
Специальность: Туризм, гостиницы, рестораны : 16, 84, собственные числа: 4.5614705085754395
Специальность: Рабочий персонал : 314, 560, собственные числа: 13.550824165344238
Специальность: Информационные технологии, интернет, телеком : 8049, 5184, собственные числа: 51.4238395690918
Специальность: Безопасность : 74, 210, собственные числа: 7.137026786804199
Специальность: Бухгалтерия, управленческий учет, финансы предприятия : 68, 246, собственные числа: 6.59346342086792
Специальность: Продажи : 603, 834, собственные числа: 29.56612205505371
Специальность: Медицина, фармацевтика : 37, 163, собственные числа: 7.3853840827941895
Специальность: Искусство, развлечения, масс-медиа : 305, 591, собственные числа: 16.22970199584961
Специальность: Начало карьеры, студенты : 262, 676, собственные числа: 16.421289443969727
Специальность: Добыча сырья : 14, 73, собственные числа: 3.964

In [ ]:
for area in prof_areas:
  data = vacancies[
            vacancies['Profarea names'].apply(
                lambda prof_list: any(area in prof for prof in prof_list)
            )
        ].copy().dropna().reset_index(drop=True)

  print(f"Специальность: {area} : {len(data)}")

Специальность: Государственная служба, некоммерческие организации : 69
Специальность: Туризм, гостиницы, рестораны : 38
Специальность: Рабочий персонал : 885
Специальность: Информационные технологии, интернет, телеком : 17580
Специальность: Безопасность : 211
Специальность: Бухгалтерия, управленческий учет, финансы предприятия : 152
Специальность: Продажи : 1576
Специальность: Медицина, фармацевтика : 107
Специальность: Искусство, развлечения, масс-медиа : 629
Специальность: Начало карьеры, студенты : 739
Специальность: Добыча сырья : 28
Специальность: Домашний персонал : 13
Специальность: Высший менеджмент : 230
Специальность: Производство, сельское хозяйство : 693
Специальность: Управление персоналом, тренинги : 111
Специальность: Консультирование : 918
Специальность: Юристы : 13
Специальность: Инсталляция и сервис : 998
Специальность: Строительство, недвижимость : 446
Специальность: Автомобильный бизнес : 48
Специальность: Банки, инвестиции, лизинг : 284
Специальность: Маркетинг, ре

## Functions


In [ ]:
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

################################################################################
# Обработка датафрейма и построение матрицы навыков
################################################################################

def get_all_skills(data: pd.DataFrame) -> list:
    """
    Собирает все уникальные навыки из столбца 'Skills' (список навыков в каждой строке).
    Возвращает список навыков (строки в нижнем регистре).
    """
    all_skills = []
    for row in data['Skills']:
        # row - это список навыков для строки датафрейма
        lowercase_list = [item.lower() for item in row]
        all_skills.extend(lowercase_list)

    return list(set(all_skills))


def make_cf(indices: list, skill_amount: int) -> list:
    """
    Создаёт вектор признаков для одной строки датафрейма.
    Напр. если у строки навыки: [0, 3, 5], а всего навыков skill_amount=10,
    то возвращаем вектор длины 10, где на позициях (0,3,5) будет 1, остальные 0.
    """
    cf = [0]*skill_amount
    for i in indices:
        cf[i] = 1
    return cf


def get_matrix(data: pd.DataFrame, skill_id_dict: dict) -> list:
    """
    Строит матрицу A размера [num_rows x skill_amount],
    где num_rows = число строк в data.
    """
    skill_amount = len(skill_id_dict)
    A = []
    for row in data['Skills']:
        # Преобразуем навыки строки в индексы
        indices = [skill_id_dict[skill.lower()] for skill in row]
        A.append(make_cf(indices, skill_amount))
    return A


################################################################################
# Основная PyTorch-функция, которая оптимизирует интервальный лосс
################################################################################

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

def calc_torch_interval(A, from_col, to_col, lr=5.0, num_iterations=10000):
    """
    A: матрица признаков (list of lists или numpy), размер [N x M].
    from_col: список или массив c нижней границей зарплаты (длина N).
    to_col: список или массив c верхней границей зарплаты (длина N).
    lr: learning rate.
    num_iterations: количество итераций для оптимизации.

    Возвращает:
      - x: тензор PyTorch (веса навыков),
      - final_interval_loss: скаляр (итоговый интервальный лосс),
      - mae: скаляр (MAE относительно середины интервала).
    """
    # Переводим A и интервалы из Python-списков/NumPy-массивов в PyTorch-тензоры
    A_torch = torch.tensor(A, dtype=torch.float32, device=device)
    from_torch = torch.tensor(from_col, dtype=torch.float32, device=device)
    to_torch = torch.tensor(to_col, dtype=torch.float32, device=device)

    # Параметры модели: вектор x размером M (число навыков)
    # Инициализируем значением: 10000 + случайное число от 0 до 1000 чтобы убрать однаковые значения в x
    random_values = 10000 + 1000 * torch.rand((A_torch.shape[1]))
    random_v = random_values.to(device)
    x = torch.nn.Parameter(random_v)

    optimizer = torch.optim.Adam([x], lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.9, patience=500, verbose=False
    )

    for iteration in range(num_iterations):
        optimizer.zero_grad()

        # Предсказанная зарплата для каждой строки: y_pred = A*x
        y_pred = A_torch.matmul(x)

        # Интервальный лосс:
        # loss = среднее (max(0, from_i - y_pred_i) + max(0, y_pred_i - to_i))
        under = torch.clamp(from_torch - y_pred, min=0.0)  # штраф, если y_pred < from
        over  = torch.clamp(y_pred - to_torch, min=0.0)    # штраф, если y_pred > to
        interval_loss = torch.mean(under + over)

        #print(f"Loss {interval_loss}")

        interval_loss.backward()
        optimizer.step()
        scheduler.step(interval_loss)

    # После обучения считаем финальные предсказания
    y_pred_final = A_torch.matmul(x)

    # Финальный интервальный лосс
    final_interval_loss = interval_loss.detach().cpu().item()

    # Средняя точка интервала для каждой строки
    mid_interval = 0.5 * (from_torch + to_torch)

    # MAE относительно средней точки [from, to]
    mae = torch.mean(torch.abs(y_pred_final - mid_interval)).detach().cpu().item()

    return x, final_interval_loss, mae



## Test of all data

In [ ]:
#собираем все Profarea names
prof_lst = []
for i in vacancies['Profarea names']:
    prof_lst += i
prof_dict = {i: [] for i in set(prof_lst)}

for i, j in enumerate(vacancies['Profarea names']):
    for k in j:
        prof_dict[k].append(i)

prof_areas = list(prof_dict.keys())

In [ ]:
def build_skill_dict(vacancies, prof_areas):
    """
    Функция пробегается по каждому названию области,
    обучает модель (получаем вектор x) и формирует итоговый словарь.
    """
    skill_dict = {}

    for area in prof_areas:
        # 1. Фильтруем вакансии, относящиеся к выбранной области
        data = vacancies[
            vacancies['Profarea names'].apply(
                lambda prof_list: any(area in prof for prof in prof_list)
            )
        ].copy().dropna().reset_index(drop=True)

        # 2. Минимальная предобработка (как в вашем коде):
        data = data[['Skills', 'From', 'To']].rename(columns={'From': 'from', 'To': 'to'})
        # Убираем строки, где from == to
        data = data[data['from'] != data['to']]
        # Убираем строки, где Skills пуст
        data = data[data['Skills'].apply(len) > 0]

        if len(data) == 0:
            # Если для этой области данных нет, создаём пустой словарь
            skill_dict[area] = {}
            continue

        # 3. Собираем список всех навыков
        all_skills = get_all_skills(data)

        # 4. Создаём словарь маппинга навыка -> индекс
        skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

        # 5. Собираем матрицу A
        A = get_matrix(data, skill_id_dict)

        # 6. Оптимизатор возвращает вектор x — стоимости навыков
        # (final_interval_loss, mae можно сохранить при необходимости)
        x, final_interval_loss, mae = calc_torch_interval(
            A,
            data['from'].values,
            data['to'].values,
            lr=5,
            num_iterations=50000
        )

        # 7. Формируем словарь навыков и их стоимости для данной области
        area_skills = {}
        for skill, idx in skill_id_dict.items():
            # x[idx] может быть тензором или numpy.float;
            # приводим к float для удобства
            area_skills[skill] = float(x[idx])

        # 8. Сохраняем результат в общий словарь
        skill_dict[area] = area_skills

    return skill_dict



# Собираем большой словарь с навыками и их стоимостью для всех областей
skill_dict = build_skill_dict(vacancies, prof_areas)

# Теперь в result_skill_dict у вас пять ключей (пять профессиональных областей),
# в каждом — вложенный словарь "навык" -> "стоимость"

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-pack

In [ ]:
skill_dict.keys()

dict_keys(['Информационные технологии, интернет, телеком', 'Закупки', 'Административный персонал', 'Добыча сырья', 'Высший менеджмент', 'Страхование', 'Консультирование', 'Продажи', 'Безопасность', 'Юристы', 'Рабочий персонал', 'Маркетинг, реклама, PR', 'Наука, образование', 'Транспорт, логистика', 'Банки, инвестиции, лизинг', 'Туризм, гостиницы, рестораны', 'Домашний персонал', 'Медицина, фармацевтика', 'Производство, сельское хозяйство', 'Инсталляция и сервис', 'Спортивные клубы, фитнес, салоны красоты', 'Автомобильный бизнес', 'Строительство, недвижимость', 'Начало карьеры, студенты', 'Государственная служба, некоммерческие организации', 'Искусство, развлечения, масс-медиа', 'Управление персоналом, тренинги', 'Бухгалтерия, управленческий учет, финансы предприятия'])

In [ ]:
def save_skill_dict_to_json(skill_dict, filename="/content/drive/MyDrive/Colab Notebooks/cost_of_skills/skill_dict.json"):
    """
    Saves the given skill_dict to a JSON file.

    :param skill_dict: Dictionary of professional areas
                       and their corresponding skills and costs.
    :param filename: The name of the JSON file to save.
    """
    with open(filename, "w", encoding="utf-8") as f:
        # ensure_ascii=False allows saving Cyrillic (and other non-ASCII) characters properly
        json.dump(skill_dict, f, ensure_ascii=False, indent=2)
    print(f"Skill dictionary saved to {filename}")

In [ ]:
save_skill_dict_to_json(skill_dict, "/content/drive/MyDrive/Colab Notebooks/cost_of_skills/my_skill_dict.json")

Skill dictionary saved to /content/drive/MyDrive/Colab Notebooks/cost_of_skills/my_skill_dict.json


## Gradio run

In [4]:
with open("/content/drive/MyDrive/Colab Notebooks/cost_of_skills/my_skill_dict_cleaned_v2.json", "r", encoding="utf-8") as f:
        skill_dict = json.load(f)


In [5]:
def update_skills(selected_profession):
    """
    Функция вызывается при выборе профессиональной области,
    чтобы обновить список навыков в чекбоксах.
    """
    if selected_profession in skill_dict:
        new_choices = list(skill_dict[selected_profession].keys())
    else:
        new_choices = []
    return gr.update(choices=new_choices, value=[])

def calculate_skills(selected_profession, selected_skills):
    """
    Функция расчёта стоимости выбранных навыков.
    Возвращает DataFrame с названием навыка и его стоимостью.
    В конце добавляется строка с суммарной стоимостью.
    """
    # Если пользователь не выбрал профессию или навыки
    if not selected_profession or not selected_skills:
        return pd.DataFrame(columns=["Навык", "Стоимость"])

    # Берём словарь навыков для выбранной профессии
    skills_for_profession = skill_dict.get(selected_profession, {})

    # Формируем данные только по выбранным навыкам
    selected_data = {skill: skills_for_profession[skill] for skill in selected_skills}

    df = pd.DataFrame(selected_data.items(), columns=["Навык", "Стоимость"])
    total_cost = df["Стоимость"].sum()
    df.loc[len(df)] = ["Суммарная стоимость", total_cost]
    return df

In [6]:
with gr.Blocks() as demo:
    gr.Markdown("## Калькулятор стоимости навыков")

    # Выбор профессиональной области
    profession_dropdown = gr.Dropdown(
        choices=list(skill_dict.keys()),
        label="Выберите профессиональную область",
        value=None,
        interactive=True
    )

    # Чекбоксы для навыков (изначально пустые — заполняются при смене области)
    skills_checkbox = gr.CheckboxGroup(
        choices=[],
        label="Выберите навыки",
        value=[],
        interactive=True
    )

    # Кнопка для расчёта
    calc_button = gr.Button("Рассчитать стоимость")

    # Таблица результата
    result_table = gr.Dataframe(
        label="Таблица стоимости",
        headers=["Навык", "Стоимость"],
        interactive=False
    )

    # Обновление списка навыков при смене профессиональной области
    profession_dropdown.change(
        fn=update_skills,
        inputs=profession_dropdown,
        outputs=skills_checkbox
    )

    # Нажатие кнопки вызывает функцию calculate_skills
    calc_button.click(
        fn=calculate_skills,
        inputs=[profession_dropdown, skills_checkbox],
        outputs=result_table
    )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aca74b6749733601c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
